In [1]:

import pandas as pd
import numpy as np
from datetime import datetime
import re

In [2]:
def clean_sales_data(file_path):
    """
    clean sales records
    """
    
    # Read the data
    print("Reading data...")
    sales_df = pd.read_excel(file_path, sheet_name='Sales')
    store_df = pd.read_excel(file_path, sheet_name='Store')
    
    print(f"Sales data shape: {sales_df.shape}")
    print(f"Sales columns: {list(sales_df.columns)}")
    print(f"Store data shape: {store_df.shape}")
    print(f"Store columns: {list(store_df.columns)}")
    
    # Create a copy for cleaning
    cleaned_df = sales_df.copy()
    
    # 1. DATE CLEANING
    # Clean SellDate (original format YYYYMMDD)
    print("🔵 Cleaning date data...")
    def parse_mixed_date(date_str):
        if pd.isna(date_str):
            return None
        # If float (e.g., 20230521.0), convert to int first
        if isinstance(date_str, float) and date_str.is_integer():
            date_str = str(int(date_str))
        else:
            date_str = str(date_str).strip()
        # print(f"Parsing date: {date_str}")
        # Only treat as YYYYMMDD if it's exactly 8 digits (like '20230521')
        # errors='coerce' will convert invalid formats to NaT
        if len(date_str) == 8 and date_str.isdigit():
            # print(f"Parsing date as YYYYMMDD: {date_str}")
            return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')
        
        # Otherwise let pandas infer the format (e.g., '9/3/2022')
        return pd.to_datetime(date_str, errors='coerce')
    
    cleaned_df['SellDate'] = cleaned_df['SellDate'].apply(parse_mixed_date)
    
    # Clean ReceiveDate (original format M/D/YYYY)
    cleaned_df['ReceiveDate'] = cleaned_df['ReceiveDate'].apply(parse_mixed_date)
    # 2. FINANCIAL DATA CLEANING
    print("🔵 Cleaning financial data...")
    try:
        cleaned_df['Cost'] = (
            cleaned_df['Cost']
            .astype(str)              
            .str.strip()                 
            .replace(r'^$', np.nan, regex=True)
            .replace('[\$,]', '', regex=True)
            .astype(float)
        )
        cleaned_df['Revenue'] = (
            cleaned_df['Revenue']
            .astype(str)              
            .str.strip()                 
            .replace(r'^$', np.nan, regex=True)
            .replace('[\$,]', '', regex=True)
            .astype(float)
        )
    except Exception as e:
        print(f"⚠️  Financial data cleaning issue: {e}")

    
    # Calculate derived financial metrics
    cleaned_df['Profit'] = cleaned_df['Revenue'] - cleaned_df['Cost']
    cleaned_df['ProfitMargin'] = np.where(
        (cleaned_df['Revenue'] > 0) & (cleaned_df['Profit'].notna()),
        (cleaned_df['Profit'] / cleaned_df['Revenue']) * 100,
        np.nan
    )

    cleaned_df['ROI'] = np.where(
        (cleaned_df['Cost'] > 0) & (cleaned_df['Profit'].notna()),
        (cleaned_df['Profit'] / cleaned_df['Cost']) * 100,
        np.nan
    )
    
    # 3. PRODUCT LINE AND WEIGHT EXTRACTION
    print("🔵 Extracting product information...")
    
    def extract_product_info(model_name):
        """Extract product line and weight from ModelName"""
        if pd.isna(model_name):
            return None, None
        
        # Use regex to extract letters (product line) and numbers (weight)
        valid_product_lines = {'SY', 'SCA', 'TH'}
        match = re.match(r'^([A-Z]+)(\d+)', str(model_name).upper())
        if match:
            product_line = match.group(1)
            weight = int(match.group(2))
            # Validate product line
            if product_line in valid_product_lines:
                return product_line, weight
            else:
                # Invalid product line - return None for both
                return None, None
        return None, None
    
    # Extract product line and weight
    product_info = cleaned_df['ModelName'].apply(extract_product_info)
    cleaned_df['ProductLine'] = [info[0] for info in product_info]
    cleaned_df['Weight'] = [info[1] for info in product_info]
    
    # 4. WEIGHT CLASS CATEGORIZATION
    print("🔵 Categorizing weight classes...")
    
    def categorize_weight(weight):
        """Categorize products by weight class"""
        if pd.isna(weight):
            return None
        try:
            weight = float(weight)
        except (ValueError, TypeError):
            return None
        if weight <= 35:
            return 'Small'
        elif weight < 155:
            return 'Medium'
        else:
            return 'Large'
    
    cleaned_df['WeightClass'] = cleaned_df['Weight'].apply(categorize_weight)
    
    # 5. LOCATION PARSING
    print("🔵 Parsing location data...")
    
    def parse_location(location):
        """Parse location into county and state"""
        if pd.isna(location):
            return None, None
        
        parts = str(location).split(',')
        if len(parts) >= 2:
            county = parts[0].strip()
            state = parts[1].strip()
            return county, state
        return location.strip(), None
    
    location_info = cleaned_df['SaleLocation'].apply(parse_location)
    cleaned_df['City'] = [info[0] for info in location_info]
    cleaned_df['State'] = [info[1] for info in location_info]
    
    # 6. MERGE WITH STORE DATA
    print("🔵 Merging with store information...")
    initial_row_count = len(cleaned_df)
    # Merge store information
    cleaned_df = cleaned_df.merge(store_df, on='StoreID', how='left')
    print(f"  Store data merged. Records with store info: {cleaned_df['StoreName'].notna().sum()}")
    # Validate the merge
    final_row_count = len(cleaned_df)
    rows_without_store_info = cleaned_df['StoreName'].isna().sum()

    # Show which StoreIDs don't have matches
    if rows_without_store_info > 0:
        print("  ⚠️  StoreIDs without matching store information:")
        unmatched_ids = cleaned_df[cleaned_df['StoreName'].isna()]['StoreID'].unique()
        for store_id in sorted(unmatched_ids)[:10]:  # Show first 10
            count = (cleaned_df['StoreID'] == store_id).sum()
            print(f"    StoreID '{store_id}': {count} records")
        if len(unmatched_ids) > 10:
            print(f"    ... and {len(unmatched_ids) - 10} more StoreIDs")

    # Validation check
    if final_row_count != initial_row_count:
        print("  ⚠️  WARNING: Row count changed during merge! This shouldn't happen with left join.")
    else:
        print("  ✅ Merge completed successfully - no rows lost")
    # 7. INVENTORY METRICS
    print("🔵 Calculating inventory metrics...")
    
    # Calculate days in inventory
    cleaned_df['InventoryDays'] = (
        cleaned_df['SellDate'] - cleaned_df['ReceiveDate']
    ).dt.days
    
    # Handle negative inventory days (data quality issue)
    cleaned_df['InventoryDays'] = np.where(
        cleaned_df['InventoryDays'] < 0, 
        np.nan, 
        cleaned_df['InventoryDays']
    )
    
    # 8. DATA TYPE CONVERSIONS
    print("🔵 Converting data types...")
    
    cleaned_df['StoreID'] = pd.to_numeric(cleaned_df['StoreID'], errors='coerce')
    cleaned_df['SerialNumber'] = cleaned_df['SerialNumber'].astype(str)
    
    # Check for duplicates
    # 9. Check for duplicate serial numbers rows
    print("🔵 Checking for duplicate rows...")
    key_columns = ['SellDate', 'SerialNumber', 'ModelName', 'ReceiveDate', 'Cost', 'Revenue', 'StoreID', 'SaleLocation', 'Usage']

    # Step 1: Identify all rows with duplicate serial numbers
    has_duplicate_serial = cleaned_df.duplicated(subset=['SerialNumber'], keep=False)
    total_rows_with_dup_serials = has_duplicate_serial.sum()

    # Step 2: Identify exact duplicates (same across all key columns) - mark ALL instances
    all_exact_duplicates = cleaned_df.duplicated(subset=key_columns, keep=False)

    # Step 3: Exact duplicates to remove (keep first)
    exact_duplicates_to_remove = cleaned_df.duplicated(subset=key_columns, keep='first')
    num_exact_duplicates = exact_duplicates_to_remove.sum()

    # Step 4: TRUE conflicting duplicates = rows with duplicate serials that are NOT part of exact duplicate sets
    conflicting_duplicates_mask = has_duplicate_serial & ~all_exact_duplicates
    num_conflicting_rows = conflicting_duplicates_mask.sum()

    # Export for review
    exact_duplicate_rows = cleaned_df[exact_duplicates_to_remove]
    exact_duplicate_rows.to_csv("exact_duplicates_to_remove.csv", index=False)

    conflicting_duplicate_rows = cleaned_df[conflicting_duplicates_mask]
    conflicting_duplicate_rows.to_csv("conflicting_duplicates.csv", index=False)

    print("  🔄 Exported exact and conflicting duplicates for review.")

    # Drop exact duplicates
    cleaned_df = cleaned_df[~exact_duplicates_to_remove]

    # Summary statistics
    unique_serial_numbers_with_conflicts = cleaned_df[cleaned_df.duplicated(subset=['SerialNumber'], keep=False)]['SerialNumber'].nunique()

    print(f"  📊 DUPLICATE ANALYSIS:")
    print(f"     • Total rows with duplicate serial numbers: {total_rows_with_dup_serials}")
    print(f"     • Rows in exact duplicate sets (all instances): {all_exact_duplicates.sum()}")
    print(f"     • Exact duplicate rows removed (keeping first): {num_exact_duplicates}")
    print(f"     • TRUE conflicting duplicate rows: {num_conflicting_rows}")
    print(f"     • Unique serial numbers with conflicts: {unique_serial_numbers_with_conflicts}")
    print(f"     • Rows remaining after cleanup: {len(cleaned_df)}")

    # Additional validation
    total_exact_duplicate_instances = all_exact_duplicates.sum()
    expected_total = total_exact_duplicate_instances + num_conflicting_rows

    if expected_total != total_rows_with_dup_serials:
        print("  ⚠️  WARNING: Duplicate counts don't add up correctly!")
        print(f"     Expected: {expected_total}, Actual: {total_rows_with_dup_serials}")
    else:
        print("  ✅ Duplicate counts validated successfully")

    # 10. CREATE ADDITIONAL ANALYTICAL COLUMNS
    print("\n🔵 Creating additional analytical columns...")
    
    # Revenue quartiles
    cleaned_df['RevenueQuartile'] = pd.qcut(
        cleaned_df['Revenue'], 
        q=4, 
        labels=['Q1', 'Q2', 'Q3', 'Q4']
    )
    
    
    # 11. DATA QUALITY CHECKS
    print("🔵 Data quality summary:")
    
    # Check for missing values
    missing_values = cleaned_df.isnull().sum()
    print("  📊 Missing values per column:")
    for col, missing in missing_values.items():
        if missing > 0:
            print(f"    {col}: {missing} ({missing/len(cleaned_df)*100:.1f}%)")
          
    # rows with any missing or unknown values
    rows_with_null = cleaned_df.isnull().any(axis=1)
    percent_with_any_missing = rows_with_null / len(cleaned_df) * 100
    print(f"  📊 Percentage of Rows with null values: {rows_with_null.sum()} ({rows_with_null.sum()/len(cleaned_df)*100:.1f}%)")

    
    # Check for negative financial values
    negative_revenue = (cleaned_df['Revenue'] < 0).sum()
    negative_cost = (cleaned_df['Cost'] < 0).sum()
    negative_profit = (cleaned_df['Profit'] < 0).sum()
    print(f"  🔴 Negative revenue records: {negative_revenue}")
    print(f"  🔴 Negative cost records: {negative_cost}")
    print(f"  🔴 Negative profit records: {negative_profit}\n")
    
    # 11. FINAL DATASET ORGANIZATION
    print("🔵 Organizing final dataset...")
    
    # Reorder columns for better organization
    column_order = [
        # Identifiers
        'SerialNumber', 'ModelName', 'ProductLine', 'Weight', 'WeightClass',
        
        # Dates
        'SellDate', 'ReceiveDate', 'InventoryDays',
        
        # Financial
        'Cost', 'Revenue', 'Profit', 'ProfitMargin', 'ROI', 'RevenueQuartile',
        
        # Location & Store
        'StoreID', 'StoreName', 'State', 'City', 'SaleLocation',
        
        # Customer
        'Usage'
    ]
    
    # Include only columns that exist in the dataframe
    final_columns = [col for col in column_order if col in cleaned_df.columns]
    cleaned_df = cleaned_df[final_columns]
    
    print(f"Final dataset shape: {cleaned_df.shape}")
    
    return cleaned_df


In [3]:

def generate_data_summary(df):
    """Generate comprehensive data summary for analysis"""
    
    print("\n" + "="*60)
    print("🔵 DATA SUMMARY FOR TABLEAU ANALYSIS")
    print("="*60)
    
    # Basic statistics
    print(f"\n🔵 Dataset Overview:")
    print(f"Total Records: {len(df):,}")
    print(f"Date Range: {df['SellDate'].min()} to {df['SellDate'].max()}")
    print(f"Total Revenue: ${df['Revenue'].sum():,.2f}")
    print(f"Total Profit: ${df['Profit'].sum():,.2f}")
    print(f"Average Profit Margin: {df['ProfitMargin'].mean():.2f}%")
    
    # Product line analysis
    print(f"\n🏭 Product Line Breakdown:")
    product_summary = df.groupby('ProductLine').agg({
        'Revenue': ['count', 'sum', 'mean'],
        'Profit': ['sum','mean'],
        'ProfitMargin': 'mean'
    }).round(2)
    print(product_summary)
    
    # Weight class analysis
    print(f"\n⚖️ Weight Class Distribution:")
    weight_summary = df.groupby('WeightClass').agg({
        'Revenue': ['count', 'sum', 'mean'],
        'Profit': ['sum','mean'],
        'ProfitMargin': 'mean'
    }).round(2)
    print(weight_summary)
    # # Store performance analysis
    print(f"\n🏪 Store Performance:")
    store_summary = df.groupby(['StoreID', 'StoreName']).agg({
        'Revenue': ['count', 'sum', 'mean'],
        'Profit': ['sum','mean'],
        'ProfitMargin': 'mean'
    }).round(2)
    print(store_summary)
    # # Geographic distribution
    print(f"\n🗺️ Top 10 States by Revenue:")
    state_summary = df.groupby('State').agg({
        'Revenue': ['count', 'sum', 'mean'],
        'Profit': ['sum','mean'],
        'ProfitMargin': 'mean'
    }).round(2).sort_values(('Revenue', 'sum'), ascending=False).head(10)

    print(state_summary)
    # # Usage category analysis
    print(f"\n👥 Usage Category Performance:")
    usage_summary = df.groupby('Usage').agg({
        'Revenue': ['count', 'sum', 'mean'],
        'Profit': ['sum','mean'],
        'ProfitMargin': 'mean'
    }).round(2)
    print(usage_summary)



In [ ]:
def main():
    file_path = 'BA_DummyData_2025.xlsx' 
    
    try:
        # Clean the data
        cleaned_data = clean_sales_data(file_path)
        
        # Generate summary
        generate_data_summary(cleaned_data)
        
        # Save cleaned data
        output_file = 'cleaned_BA_DummyData_2025.csv'
        cleaned_data.to_csv(output_file, index=False)
        print(f"\n✅ Cleaned data saved to: {output_file}")
        
        return cleaned_data
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        print("Please update the file_path variable with the correct path to your CSV file")
        return None
    except Exception as e:
        print(f"❌ Error processing data: {str(e)}")
        return None

In [6]:
    
    # # Inventory performance categories
    # def categorize_inventory_performance(days):
    #     if pd.isna(days):
    #         return 'Unknown'
    #     elif days <= 90:
    #         return 'Fast'
    #     elif days <= 180:
    #         return 'Normal'
    #     elif days <= 365:
    #         return 'Slow'
    #     else:
    #         return 'Very Slow'
    
    # cleaned_df['InventoryPerformance'] = cleaned_df['InventoryDays'].apply(
    #     categorize_inventory_performance
    # )